In [1]:
#Hello_world of Machine Learning
import pandas as pd

In [2]:
#converting csv to dataframe
shared_articles = pd.read_csv('shared_articles.csv')
shared_articles.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


In [3]:
user_interaction = pd.read_csv('users_interactions.csv')
user_interaction.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


In [4]:
#dropping features that we will not be using
articles = shared_articles.drop(['timestamp','authorPersonId','authorSessionId','authorUserAgent','authorRegion','authorCountry','contentType','url'], axis=1)

In [5]:
articles.head()

,eventType,contentId,title,text,lang
0,CONTENT REMOVED,-6451309518266745024,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,CONTENT SHARED,-4110354420726924665,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,CONTENT SHARED,-7292285110016212249,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,CONTENT SHARED,-6151852268067518688,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,CONTENT SHARED,2448026894306402386,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


In [6]:
articles['eventType'].value_counts()

CONTENT SHARED     3047
CONTENT REMOVED      75
Name: eventType, dtype: int64

In [7]:
articles['lang'].value_counts()

en    2264
pt     850
la       4
es       2
ja       2
Name: lang, dtype: int64

In [8]:
#content that was removed may create a bias
articles.drop(articles[articles.eventType == 'CONTENT REMOVED'].index, inplace = True)



In [9]:
articles.count()

eventType    3047
contentId    3047
title        3047
text         3047
lang         3047
dtype: int64

In [10]:
#dropping features from next data source
activity = user_interaction.drop(['timestamp','personId','sessionId','userAgent','userRegion','userCountry'], axis=1)

In [11]:
activity.head()

,eventType,contentId
0,VIEW,-3499919498720038879
1,VIEW,8890720798209849691
2,VIEW,310515487419366995
3,FOLLOW,310515487419366995
4,VIEW,-7820640624231356730


In [12]:
#count of contentId doesn't match
n = len(pd.unique(activity['contentId']))
print(n)

2987


In [13]:
activity.count()

eventType    72312
contentId    72312
dtype: int64

In [14]:
activity['eventType'].value_counts()

VIEW               61086
LIKE                5745
BOOKMARK            2463
COMMENT CREATED     1611
FOLLOW              1407
Name: eventType, dtype: int64

In [15]:
#pivoting to join
activity['COUNTER'] =1       #initially, set that counter to 1.
group_activity = activity.groupby(['contentId','eventType'])['COUNTER'].sum().reset_index() #sum function

In [16]:
events = group_activity.pivot_table('COUNTER', ['contentId'], 'eventType')

In [17]:
events

eventType,BOOKMARK,COMMENT CREATED,FOLLOW,LIKE,VIEW
contentId,,,,,
-9222795471790223670,1.0,2.0,3.0,4.0,16.0
-9216926795620865886,1.0,1.0,1.0,3.0,15.0
-9194572880052200111,2.0,1.0,1.0,4.0,21.0
-9192549002213406534,NaN,1.0,NaN,5.0,50.0
-9190737901804729417,NaN,NaN,NaN,1.0,8.0
-9189659052158407108,2.0,NaN,NaN,1.0,33.0
-9184137057748005562,NaN,NaN,NaN,NaN,1.0
-9176143510534135851,2.0,NaN,NaN,8.0,23.0
-9172673334835262304,NaN,NaN,NaN,NaN,1.0


In [18]:
events = events.fillna(0)

In [19]:
def label(row):
    return (1* row['VIEW']) + (4*row['LIKE']) + (10*row['COMMENT CREATED']) +( 25*row['FOLLOW'] )+ (100*row['BOOKMARK'])

events['virality'] = events.apply (lambda row: label(row), axis=1)
     

In [20]:
events

eventType,BOOKMARK,COMMENT CREATED,FOLLOW,LIKE,VIEW,virality
contentId,,,,,,
-9222795471790223670,1.0,2.0,3.0,4.0,16.0,227.0
-9216926795620865886,1.0,1.0,1.0,3.0,15.0,162.0
-9194572880052200111,2.0,1.0,1.0,4.0,21.0,272.0
-9192549002213406534,0.0,1.0,0.0,5.0,50.0,80.0
-9190737901804729417,0.0,0.0,0.0,1.0,8.0,12.0
-9189659052158407108,2.0,0.0,0.0,1.0,33.0,237.0
-9184137057748005562,0.0,0.0,0.0,0.0,1.0,1.0
-9176143510534135851,2.0,0.0,0.0,8.0,23.0,255.0
-9172673334835262304,0.0,0.0,0.0,0.0,1.0,1.0


In [21]:
articles


,eventType,contentId,title,text,lang
1,CONTENT SHARED,-4110354420726924665,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,CONTENT SHARED,-7292285110016212249,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,CONTENT SHARED,-6151852268067518688,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,CONTENT SHARED,2448026894306402386,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
5,CONTENT SHARED,-2826566343807132236,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en
6,CONTENT SHARED,-2148899391355011268,Banks Need To Collaborate With Bitcoin and Fin...,It will take time until banks come around to t...,en
7,CONTENT SHARED,4119190424078847945,Blockchain Technology Could Put Bank Auditors ...,When most people think about computers and rob...,en
8,CONTENT SHARED,-7926018713416777892,Why Decentralized Conglomerates Will Scale Bet...,"Bitcoin.com spoke with the OpenLedger CEO, Ron...",en
9,CONTENT SHARED,3353902017498793780,The Rise And Growth of Ethereum Gets Mainstrea...,"Ethereum, considered by many to be the most pr...",en
10,CONTENT SHARED,-9157338616628196758,Situação financeira ruim de varejistas pressio...,A queda nas vendas e a deterioração na situaçã...,pt


In [22]:
print(articles.text[1])

All of this work is still very early. The first full public version of the Ethereum software was recently released, and the system could face some of the same technical and legal problems that have tarnished Bitcoin. Many Bitcoin advocates say Ethereum will face more security problems than Bitcoin because of the greater complexity of the software. Thus far, Ethereum has faced much less testing, and many fewer attacks, than Bitcoin. The novel design of Ethereum may also invite intense scrutiny by authorities given that potentially fraudulent contracts, like the Ponzi schemes, can be written directly into the Ethereum system. But the sophisticated capabilities of the system have made it fascinating to some executives in corporate America. IBM said last year that it was experimenting with Ethereum as a way to control real world objects in the so-called Internet of things. Microsoft has been working on several projects that make it easier to use Ethereum on its computing cloud, Azure. "Eth

In [23]:
# remove special characters, numbers, punctuations
articles['text'] = articles['text'].str.replace("[^a-zA-Z#]", " ")

In [24]:
print(articles.text[1])

All of this work is still very early  The first full public version of the Ethereum software was recently released  and the system could face some of the same technical and legal problems that have tarnished Bitcoin  Many Bitcoin advocates say Ethereum will face more security problems than Bitcoin because of the greater complexity of the software  Thus far  Ethereum has faced much less testing  and many fewer attacks  than Bitcoin  The novel design of Ethereum may also invite intense scrutiny by authorities given that potentially fraudulent contracts  like the Ponzi schemes  can be written directly into the Ethereum system  But the sophisticated capabilities of the system have made it fascinating to some executives in corporate America  IBM said last year that it was experimenting with Ethereum as a way to control real world objects in the so called Internet of things  Microsoft has been working on several projects that make it easier to use Ethereum on its computing cloud  Azure   Eth

In [25]:
#removing stop words
articles['text'] = articles['text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [26]:
print(articles.text[1])

this work still very early first full public version Ethereum software recently released system could face some same technical legal problems that have tarnished Bitcoin Many Bitcoin advocates Ethereum will face more security problems than Bitcoin because greater complexity software Thus Ethereum faced much less testing many fewer attacks than Bitcoin novel design Ethereum also invite intense scrutiny authorities given that potentially fraudulent contracts like Ponzi schemes written directly into Ethereum system sophisticated capabilities system have made fascinating some executives corporate America said last year that experimenting with Ethereum control real world objects called Internet things Microsoft been working several projects that make easier Ethereum computing cloud Azure Ethereum general platform where solve problems many industries using fairly elegant solution most elegant solution have seen date said Marley Gray director business development strategy Microsoft Gray respo

In [27]:
# remove special characters, numbers, punctuations
articles['title'] = articles['title'].str.replace("[^a-zA-Z#]", " ")
articles['title'] = articles['title'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [28]:
articles.drop(articles[articles.lang != 'en'].index, inplace = True)

In [29]:
articles.head()

,eventType,contentId,title,text,lang
1,CONTENT SHARED,-4110354420726924665,Ethereum Virtual Currency Enables Transactions...,this work still very early first full public v...,en
2,CONTENT SHARED,-7292285110016212249,Bitcoin Future When GBPcoin Branson Wins Over ...,alarm clock wakes with stream advert free broa...,en
3,CONTENT SHARED,-6151852268067518688,Google Data Center Tour,excited share Google Data Center Tour YouTube ...,en
4,CONTENT SHARED,2448026894306402386,Wants Evolve Internet With Blockchain Technology,Aite Group projects blockchain market could va...,en
5,CONTENT SHARED,-2826566343807132236,IEEE Talk Blockchain Cloud Computing Oxford Co...,largest oldest organizations computing profess...,en


In [34]:
#lets join
articles.head()

,eventType,contentId,title,text,lang
1,CONTENT SHARED,-4110354420726924665,Ethereum Virtual Currency Enables Transactions...,this work still very early first full public v...,en
2,CONTENT SHARED,-7292285110016212249,Bitcoin Future When GBPcoin Branson Wins Over ...,alarm clock wakes with stream advert free broa...,en
3,CONTENT SHARED,-6151852268067518688,Google Data Center Tour,excited share Google Data Center Tour YouTube ...,en
4,CONTENT SHARED,2448026894306402386,Wants Evolve Internet With Blockchain Technology,Aite Group projects blockchain market could va...,en
5,CONTENT SHARED,-2826566343807132236,IEEE Talk Blockchain Cloud Computing Oxford Co...,largest oldest organizations computing profess...,en


In [35]:
events.head()

,contentId,BOOKMARK,COMMENT CREATED,FOLLOW,LIKE,VIEW,virality
0,-9222795471790223670,1.0,2.0,3.0,4.0,16.0,227.0
1,-9216926795620865886,1.0,1.0,1.0,3.0,15.0,162.0
2,-9194572880052200111,2.0,1.0,1.0,4.0,21.0,272.0
3,-9192549002213406534,0.0,1.0,0.0,5.0,50.0,80.0
4,-9190737901804729417,0.0,0.0,0.0,1.0,8.0,12.0


In [36]:
#actual magic happens here
joined_data = articles.join(events.set_index('contentId'), on="contentId")

In [37]:
joined_data.head()

,eventType,contentId,title,text,lang,BOOKMARK,COMMENT CREATED,FOLLOW,LIKE,VIEW,virality
1,CONTENT SHARED,-4110354420726924665,Ethereum Virtual Currency Enables Transactions...,this work still very early first full public v...,en,0.0,0.0,0.0,0.0,1.0,1.0
2,CONTENT SHARED,-7292285110016212249,Bitcoin Future When GBPcoin Branson Wins Over ...,alarm clock wakes with stream advert free broa...,en,0.0,0.0,0.0,0.0,1.0,1.0
3,CONTENT SHARED,-6151852268067518688,Google Data Center Tour,excited share Google Data Center Tour YouTube ...,en,0.0,0.0,0.0,3.0,10.0,22.0
4,CONTENT SHARED,2448026894306402386,Wants Evolve Internet With Blockchain Technology,Aite Group projects blockchain market could va...,en,NaN,NaN,NaN,NaN,NaN,NaN
5,CONTENT SHARED,-2826566343807132236,IEEE Talk Blockchain Cloud Computing Oxford Co...,largest oldest organizations computing profess...,en,0.0,0.0,0.0,0.0,2.0,2.0


In [38]:
joined_data['LIKE'].isnull().sum()

62

In [39]:
joined_data['BOOKMARK'].isnull().sum()

62

In [40]:
#removing content with no virality information
joined_data = joined_data.dropna()

In [41]:
joined_data['LIKE'].count()

2149

In [42]:
#developing experimental models
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(joined_data['title'])

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# TF-IDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(articles['title'])

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np

train_bow = bow[:2149,:]

# splitting data into training and test set
xtrain_bow, xtest_bow, ytrain, ytest = train_test_split(train_bow, joined_data['virality'], random_state=42, test_size=0.3)



In [45]:
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score


lr = LinearRegression()
lr.fit(xtrain_bow, ytrain)
lr.score(xtrain_bow, ytrain)
predict_test = lr.predict(xtest_bow)


res = dict()
metrics = dict()

res['lr'] = lr.coef_
metrics['lr'] = r2_score(ytest, predict_test),explained_variance_score(ytest, predict_test)

In [46]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(xtrain_bow, ytrain)
predict_clf = clf.predict(xtest_bow)
res['lasso'] = clf.coef_
metrics['lasso'] = r2_score(ytest, predict_clf),explained_variance_score(ytest, predict_clf)

In [47]:
for m in metrics.items():
    print(m)

('lr', (-10.676099523743668, -10.613439308272504))
('lasso', (-1.5228739761557, -1.5193597937856884))


In [48]:
train_tfidf = tfidf[:2149,:]

# splitting data into training and validation set
xtrain_tfidf, xtest_tfidf, ytrain, ytest = train_test_split(train_tfidf, joined_data['virality'], random_state=42, test_size=0.3)



In [49]:
lr = LinearRegression()
lr.fit(xtrain_tfidf, ytrain)
lr.score(xtrain_tfidf, ytrain)
predict_test = lr.predict(xtest_tfidf)


res['lr-tfidf'] = lr.coef_
metrics['lr-tfidf'] = r2_score(ytest, predict_test),explained_variance_score(ytest, predict_test)

In [50]:
clf = linear_model.Lasso(alpha=0.1)
clf.fit(xtrain_tfidf, ytrain)
predict_clf = clf.predict(xtest_tfidf)
res['lasso-tfidf'] = clf.coef_
metrics['lasso-tfidf'] = r2_score(ytest, predict_clf),explained_variance_score(ytest, predict_clf)

In [51]:
for m in metrics.items():
    print(m)

('lr', (-10.676099523743668, -10.613439308272504))
('lasso', (-1.5228739761557, -1.5193597937856884))
('lr-tfidf', (-5.4346682178149806, -5.430368651618944))
('lasso-tfidf', (-0.9591277891503331, -0.9591213609497176))
